# Bootstrapping 

In [1]:
#%reset
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn import metrics

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, confusion_matrix
from num2words import num2words
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, KFold, RepeatedStratifiedKFold
from sklearn.metrics import f1_score, matthews_corrcoef, roc_auc_score
import word2number
from word2number import w2n
from sklearn.tree import DecisionTreeClassifier
import pickle
import random

hfont = {'fontname':'Helvetica'}

Set wd to be a folder not on github

In [17]:
new_directory = '/Users/rem76/Documents/COVID_projections/Bootstrapping/'
os.chdir(new_directory)

In [7]:
def create_column_names(categories_for_subsetting, num_of_weeks):
    column_names = ['HSA_ID']

    for week in range(1, num_of_weeks + 1):
        week = num2words(week)
        for category in categories_for_subsetting:
            column_name = f'week_{week}_{category}'
            column_names.append(column_name)

    return column_names

def create_collated_weekly_data(pivoted_table, original_data, categories_for_subsetting, geography, column_names):
    collated_data = pd.DataFrame(index=range(51), columns=column_names)

    x = 0
    for geo in original_data[geography].unique():
        #matching_indices = [i for i, geo_col in enumerate(pivoted_table) if geo_col == geo]
        collated_data.loc[x, geography] = geo
        columns_to_subset = [f'{geo}_{category}' for category in categories_for_subsetting]
        j = 1
        try:
            for row in range(len(pivoted_table.loc[:, columns_to_subset])):
                collated_data.iloc[x, j:j + len(categories_for_subsetting)] = pivoted_table.loc[row, columns_to_subset]
                j += len(categories_for_subsetting)
        except:
            pass
        x += 1

    return collated_data


In [8]:
def calculate_metrics(confusion_matrix):
    # Extract values from the confusion matrix
    TP = confusion_matrix[1, 1]
    FP = confusion_matrix[0, 1]
    TN = confusion_matrix[0, 0]
    FN = confusion_matrix[1, 0]

    # Calculate Sensitivity (True Positive Rate) and Specificity (True Negative Rate)
    sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0.0

    # Calculate PPV (Precision) and NPV
    ppv = TP / (TP + FP) if (TP + FP) > 0 else 0.0
    npv = TN / (TN + FN) if (TN + FN) > 0 else 0.0

    return sensitivity, specificity, ppv, npv

In [5]:
def prep_training_test_data_period(data, no_weeks, weeks_in_future, geography, weight_col, keep_output):
## Get the weeks for the x and y datasets   
    x_weeks = []  
    y_weeks = []
    y_weeks_to_check = [] #check these weeks to see if any of them are equal to 1
    for week in no_weeks:
        test_week = int(week) + weeks_in_future
        x_weeks.append('_' + num2words(week) + '_')
        for week_y in range(week+1, test_week+1):
                y_weeks_to_check.append('_' + num2words(week_y) + '_')
        y_weeks.append('_' + num2words(test_week) + '_')
    
    X_data = pd.DataFrame()
    y_data = pd.DataFrame()
    weights_all =  pd.DataFrame()
    missing_data = []
    HSA_IDs_list = []
    ## Now get the training data 
    k = 0
    for x_week in x_weeks:
            y_week = y_weeks[k]
            k +=1

            weeks_x = [col for col in data.columns if x_week in col]
            columns_x  = [geography] + weeks_x + [weight_col]
            data_x = data[columns_x]

            weeks_y = [col for col in data.columns if y_week in col]
            columns_y  = [geography] + weeks_y
            data_y = data[columns_y]
            ### now add the final column to the y data that has it so that it's if any week in the trhee week perdiod exceeded 15
            train_week = w2n.word_to_num(x_week.replace("_", ""))
            target_week =  w2n.word_to_num(y_week.replace("_", ""))
            y_weeks_to_check = []
            for week_to_check in range(train_week + 1, target_week + 1):
                y_weeks_to_check.append('_' + num2words(week_to_check) + '_')

            y_weeks_to_check = [week + 'beds_over_15_100k' for week in y_weeks_to_check]
            columns_to_check = [col for col in data.columns if any(week in col for week in y_weeks_to_check)]
            y_over_in_period = data[columns_to_check].apply(max, axis=1)
            data_y = pd.concat([data_y, y_over_in_period], axis=1)
            # ensure they have the same amount of data
            #remove rows in test_data1 with NA in test_data2
            data_x = data_x.dropna()
            data_x = data_x[data_x[geography].isin(data_y[geography])]
            # remove rows in test_data2 with NA in test_data1
            data_y = data_y.dropna()
            data_y = data_y[data_y[geography].isin(data_x[geography])]
            data_x = data_x[data_x[geography].isin(data_y[geography])]
            data_x_no_HSA = len(data_x[geography].unique())

            missing_data.append(((len(data[geography].unique()) - data_x_no_HSA)/len(data[geography].unique())) * 100)
            # get weights 
            #weights = weight_data[weight_data[geography].isin(data_x[geography])][[geography, weight_col]]

            X_week = data_x.iloc[:, 1:len(columns_x)]  # take away y, leave weights for mo
            y_week = data_y.iloc[:, -1] 
            
            y_week = y_week.astype(int)

            weights = X_week.iloc[:, -1] 
            if keep_output:
                X_week = X_week.iloc[:, :len(X_week.columns)-1] # remove the weights and leave "target" for that week

                #rename columns for concatenation 
                X_week.columns = range(1, len(data_x.columns) -1)
            else:
                X_week = X_week.iloc[:, :len(X_week.columns)-2] # remove the weights and  "target" for that week

                X_week.columns = range(1, len(data_x.columns) -2)# remove the weights and  "target" for that week

            y_week.columns = range(1, len(data_y.columns) -2)
            X_data = pd.concat([X_data, X_week])
            y_data = pd.concat([y_data, y_week]) 
            weights_all =  pd.concat([weights_all, weights]) 
            HSA_IDs_list.append(data_x[geography].reset_index(drop=True))



    X_data.reset_index(drop=True, inplace=True)
    y_data.reset_index(drop=True, inplace=True)
    weights_all.reset_index(drop=True, inplace=True)
    HSA_IDs = pd.DataFrame({'HSA_ID': pd.concat(HSA_IDs_list, ignore_index=True)})
    HSA_IDs.reset_index(drop=True, inplace=True)
    return(X_data, y_data, weights_all, missing_data, HSA_IDs)


### this code it's ANY in the x week period 
def prep_training_test_data(data, no_weeks, weeks_in_future, geography, weight_col, keep_output):
## Get the weeks for the x and y datasets   
    x_weeks = []  
    y_weeks = []
    for week in no_weeks:
        test_week = int(week) + weeks_in_future
        x_weeks.append('_' + num2words(week) + '_')
        y_weeks.append('_' + num2words(test_week) + '_')
    
    X_data = pd.DataFrame()
    y_data = pd.DataFrame()
    weights_all =  pd.DataFrame()
    missing_data = []
    ## Now get the training data \
    k = 0
    for x_week in x_weeks:
            y_week = y_weeks[k]
            k += 1
            weeks_x = [col for col in data.columns if x_week in col]
            columns_x  = [geography] + weeks_x + [weight_col]
            data_x = data[columns_x]

            weeks_y = [col for col in data.columns if y_week in col]
            columns_y  = [geography] + weeks_y
            data_y = data[columns_y]
            # ensure they have the same amount of data
            #remove rows in test_data1 with NA in test_data2
            data_x = data_x.dropna()
            data_x = data_x[data_x[geography].isin(data_y[geography])]
            # remove rows in test_data2 with NA in test_data1
            data_y = data_y.dropna()
            data_y = data_y[data_y[geography].isin(data_x[geography])]
            data_x = data_x[data_x[geography].isin(data_y[geography])]
            data_x_no_HSA = len(data_x[geography].unique())

            missing_data.append(((len(data[geography].unique()) - data_x_no_HSA)/len(data[geography].unique())) * 100)
            # get weights 
            #weights = weight_data[weight_data[geography].isin(data_x[geography])][[geography, weight_col]]

            X_week = data_x.iloc[:, 1:len(columns_x)]  # take away y, leave weights for mo
            y_week = data_y.iloc[:, -1] 
            
            y_week = y_week.astype(int)

            weights = X_week.iloc[:, -1] 
            if keep_output:
                X_week = X_week.iloc[:, :len(X_week.columns)-1] # remove the weights and leave "target" for that week

                #rename columns for concatenation 
                X_week.columns = range(1, len(data_x.columns) -1)
            else:
                X_week = X_week.iloc[:, :len(X_week.columns)-2] # remove the weights and  "target" for that week

                X_week.columns = range(1, len(data_x.columns) -2)# remove the weights and  "target" for that week

                #rename columns for concatenation 
            y_week.columns = range(1, len(data_y.columns) -1)
            X_data = pd.concat([X_data, X_week])
            y_data = pd.concat([y_data, y_week]) 
        
            weights_all =  pd.concat([weights_all, weights]) 


    X_data.reset_index(drop=True, inplace=True)
    y_data.reset_index(drop=True, inplace=True)
    weights_all.reset_index(drop=True, inplace=True)

    return(X_data, y_data, weights_all, missing_data)

In [4]:
# This function performs cross-validation using a leave-geography-out approach with bootstrap sampling.
# It trains and evaluates a classifier on subsets of the data for multiple iterations.

def cross_validation_leave_geo_out_bootstrap(data, geography_column, geo_split, no_iterations, cv, classifier, param_grid, no_iterations_param, no_weeks_train, no_weeks_test, weeks_in_future, weight_col, keep_output, time_period):
    
    # Lists to store best hyperparameters and area under ROC curve (auROC) scores for each iteration
    best_hyperparameters_per_iter = []
    auROC_per_iter = []

    # Loop through the specified number of iterations
    for i in range(no_iterations):
        
        # Subset the HSAs (Health Service Areas) from the full dataset based on geography
        geo_names = data[geography_column].unique()
        num_names_to_select = int(geo_split * len(geo_names))
        geos_for_sample = random.sample(list(geo_names), num_names_to_select)
        subset_HSAs_for_train = data[data[geography_column].isin(geos_for_sample)]
        subset_HSAs_for_test = data[~data[geography_column].isin(geos_for_sample)]

        # Resample and prepare training data
        subset_HSAs_for_train_data_resampled = subset_HSAs_for_train.sample(frac=1, replace=True)
        subset_HSAs_for_train_data_resampled_copy = subset_HSAs_for_train_data_resampled.copy()
        weights_train = subset_HSAs_for_train_data_resampled_copy.loc[:, 'weight']
        y_sample_train = subset_HSAs_for_train_data_resampled_copy.loc[:, 'y']  # Assuming the last column is the target column
        X_sample_train = subset_HSAs_for_train_data_resampled_copy.drop(subset_HSAs_for_train_data_resampled_copy.columns[-3:], axis=1) #final three are HSA, y, and weight

        # Resample and prepare test data
        subset_HSAs_for_test_data_resampled = subset_HSAs_for_test.sample(frac=1, replace=True)
        subset_HSAs_for_test_data_resampled_copy = subset_HSAs_for_test_data_resampled.copy()
        weights_test = subset_HSAs_for_test_data_resampled_copy.loc[:, 'weight']

        y_sample_test = subset_HSAs_for_test_data_resampled_copy.loc[:, 'y']  # Assuming the last column is the target column
        X_sample_test = subset_HSAs_for_test_data_resampled_copy.drop(subset_HSAs_for_test_data_resampled_copy.columns[-3:], axis=1) # final three are HSA, y, and weight

        # Perform random search for hyperparameter tuning
        random_search = RandomizedSearchCV(classifier, param_grid, n_iter=no_iterations_param, cv=cv, random_state=10)
        random_search.fit(X_sample_train, y_sample_train, sample_weight=weights_train)
        best_params = random_search.best_params_

        # Create the Decision Tree classifier with the best hyperparameters
        model = DecisionTreeClassifier(**best_params, random_state=10, class_weight='balanced')
        model_fit = model.fit(X_sample_train, y_sample_train, sample_weight=weights_train)
        y_pred = model_fit.predict_proba(X_sample_test)

        # Evaluate the performance of the model and store results
        best_hyperparameters_per_iter.append(best_params)
        print(best_hyperparameters_per_iter)
        auROC_per_iter.append(roc_auc_score(y_sample_test, y_pred[:, 1]))
        print(roc_auc_score(y_sample_test, y_pred[:, 1]))
    # Return the best hyperparameters based on the iteration with the highest auROC score
    return best_hyperparameters_per_iter[np.argmax(np.array(auROC_per_iter))]



In [3]:
def calculate_percentiles(iterations, model_name, ROC_actual, accuracy_actual, sensitivity_actual, specificity_actual, ppv_actual, npv_actual, X_test ,y_test):
        bootstrapped_stats_ROC = []
        bootstrapped_stats_accuracy = []
        bootstrapped_stats_sesitivity = []
        bootstrapped_stats_specificity = []
        bootstrapped_stats_ppv = []
        bootstrapped_stats_npv = []

        for j in iterations:
            model_name_to_load = model_name + "_" + str(j)+ ".sav" 
            model_fit = pickle.load(open(model_name_to_load, 'rb'))
            y_bootstrap_predict = model_fit.predict(X_test)
            y_bootstrap_predict_proba = model_fit.predict_proba(X_test)

            ROC_AUC_bootstrap_test_performance = metrics.roc_auc_score(y_test, y_bootstrap_predict_proba[:,1]) 
            accuracy_bootstrap_test_performance  = accuracy_score(y_test, y_bootstrap_predict)

            sensitivity_bootstrap_test_performance, specificity_bootstrap_test_performance, ppv_bootstrap_test_performance, npv_bootstrap_test_performance = calculate_metrics(confusion_matrix(y_test, y_bootstrap_predict))
        ### (D) Calculate estimate fo variance  by getting (B) - (D) 

            bootstrapped_stats_ROC.append({'Difference': ROC_AUC_bootstrap_test_performance - ROC_actual}) ## according to https://ocw.mit.edu/courses/18-05-introduction-to-probability-and-statistics-spring-2014/resources/mit18_05s14_reading24/
            bootstrapped_stats_accuracy.append({'Difference': accuracy_bootstrap_test_performance - accuracy_actual}) ## according to https://ocw.mit.edu/courses/18-05-introduction-to-probability-and-statistics-spring-2014/resources/mit18_05s14_reading24/
            bootstrapped_stats_sesitivity.append({'Difference': sensitivity_bootstrap_test_performance - sensitivity_actual}) ## according to https://ocw.mit.edu/courses/18-05-introduction-to-probability-and-statistics-spring-2014/resources/mit18_05s14_reading24/
            bootstrapped_stats_specificity.append({'Difference': specificity_bootstrap_test_performance - specificity_actual}) ## according to https://ocw.mit.edu/courses/18-05-introduction-to-probability-and-statistics-spring-2014/resources/mit18_05s14_reading24/
            bootstrapped_stats_ppv.append({'Difference': ppv_bootstrap_test_performance - ppv_actual}) ## according to https://ocw.mit.edu/courses/18-05-introduction-to-probability-and-statistics-spring-2014/resources/mit18_05s14_reading24/
            bootstrapped_stats_npv.append({'Difference': npv_bootstrap_test_performance - npv_actual}) ## according to https://ocw.mit.edu/courses/18-05-introduction-to-probability-and-statistics-spring-2014/resources/mit18_05s14_reading24/


        bootstrapped_stats_ROC = pd.DataFrame(bootstrapped_stats_ROC)
        bootstrapped_stats_accuracy = pd.DataFrame(bootstrapped_stats_accuracy)
        bootstrapped_stats_sesitivity = pd.DataFrame(bootstrapped_stats_sesitivity)
        bootstrapped_stats_specificity = pd.DataFrame(bootstrapped_stats_specificity)
        bootstrapped_stats_ppv = pd.DataFrame(bootstrapped_stats_ppv)
        bootstrapped_stats_npv = pd.DataFrame(bootstrapped_stats_npv)

    ## Step 3: Get percentile
        alpha = 0.05

        upper_quartile_ROC, lower_quartile_ROC = ROC_actual - np.percentile(bootstrapped_stats_ROC["Difference"], [100 * (1 - alpha / 2.0), 100 * alpha / 2.0])
        upper_quartile_accuracy, lower_quartile_accuracy = accuracy_actual - np.percentile(bootstrapped_stats_accuracy["Difference"], [100 * (1 - alpha / 2.0), 100 * alpha / 2.0])
        upper_quartile_sensitivity, lower_quartile_sensitivity = sensitivity_actual - np.percentile(bootstrapped_stats_sesitivity["Difference"], [100 * (1 - alpha / 2.0), 100 * alpha / 2.0])
        upper_quartile_specificity, lower_quartile_specificity = specificity_actual - np.percentile(bootstrapped_stats_specificity["Difference"], [100 * (1 - alpha / 2.0), 100 * alpha / 2.0])
        upper_quartile_ppv, lower_quartile_ppv = ppv_actual - np.percentile(bootstrapped_stats_ppv["Difference"], [100 * (1 - alpha / 2.0), 100 * alpha / 2.0])
        upper_quartile_npv, lower_quartile_npv = npv_actual - np.percentile(bootstrapped_stats_npv["Difference"], [100 * (1 - alpha / 2.0), 100 * alpha / 2.0])
        ## Step 4: Get optimization-corrected performance

        return upper_quartile_ROC, lower_quartile_ROC, upper_quartile_accuracy, lower_quartile_accuracy, upper_quartile_sensitivity, lower_quartile_sensitivity, upper_quartile_specificity, lower_quartile_specificity, upper_quartile_ppv, lower_quartile_ppv, upper_quartile_npv, lower_quartile_npv

### now try bootstrapping w/o feature selection
iterations = 100
## DO NOT SAMPLE THE TARGET DATA
def bootstrap_no_dev(iterations, training_data, clf, param_grid, cv, geo_split, iterations_param_search, model_name, time_period, keep_output, weeks_in_future, geography, weight_col, no_iterations_CV, no_weeks_train, no_weeks_test):
      #1. Get dataset
    for j in iterations:
        # sample 

          training_data_resampled = training_data.sample(frac = 1, replace=True)

          training_data_resampled_copy = training_data_resampled.copy()
          weights_train = training_data_resampled_copy.loc[:, 'weight']
          y_sample_train = training_data_resampled_copy.loc[:, 'y']

          X_sample_train = training_data_resampled_copy.drop(['HSA_ID', 'weight', 'y'],axis=1) 

          best_params = cross_validation_leave_geo_out_bootstrap(training_data_resampled, geography_column=geography, geo_split=geo_split, no_iterations=no_iterations_CV, cv=cv, classifier=clf, param_grid=param_grid, no_iterations_param=iterations_param_search, no_weeks_train=no_weeks_train, no_weeks_test=no_weeks_test, weeks_in_future=weeks_in_future, weight_col=weight_col, keep_output=keep_output, time_period=time_period)



# Create the Decision Tree classifier with the best hyperparameters
          model = DecisionTreeClassifier(**best_params,random_state=10, class_weight='balanced')
          model_fit = model.fit(X_sample_train, y_sample_train, sample_weight=weights_train)
          path = model_fit.cost_complexity_pruning_path(X_sample_train, y_sample_train)
          ccp_alphas, impurities = path.ccp_alphas, path.impurities
          clfs = []
          for ccp_alpha in ccp_alphas:
              clf = DecisionTreeClassifier(**best_params,random_state=10, ccp_alpha=ccp_alpha,  class_weight='balanced')
              clf.fit(X_sample_train, y_sample_train,  sample_weight = weights_train )
              clfs.append(clf)

          clfs = clfs[:-1]
          ccp_alphas = ccp_alphas[:-1]
          train_scores = [roc_auc_score(y_sample_train, clf.predict_proba(X_sample_train)[:, 1]) for clf in clfs]
          model_pruned = clfs[train_scores.index(max(train_scores))]

          model_fit = model_pruned.fit(X_sample_train, y_sample_train,  sample_weight = weights_train )
          model_name_to_save = model_name + "_" + str(j)+ ".sav" 
          X_data_name = model_name + "_X_data_" + str(j) + ".csv" 
          y_data_name = model_name + "_y_data_" + str(j) + ".csv" 
          weights_data_name = model_name + "_weights_" + str(j) + ".csv" 
          
          weights_train.to_csv(weights_data_name, index=False)
          X_sample_train.to_csv(X_data_name,index=False)
          y_sample_train.to_csv(y_data_name, index=False)
          pickle.dump(model_fit, open(model_name_to_save, 'wb'))


In [9]:
def merge_and_rename_data(data1, data2, on_column, suffix1, suffix2):
    merged_data = pd.merge(data1, data2, on=on_column, suffixes=('_'+suffix1, '_'+suffix2))

    new_column_names = [col.replace(f'_{on_column}_{suffix1}', f'_{suffix1}').replace(f'_{on_column}_{suffix2}', f'_{suffix2}') for col in merged_data.columns]
    merged_data.rename(columns=dict(zip(merged_data.columns, new_column_names)), inplace=True)

    return merged_data

def pivot_data_by_HSA(data, index_column, columns_column, values_column):
    data_by_HSA = data[[index_column, columns_column, values_column]]
    pivot_table = data_by_HSA.pivot_table(index=index_column, columns=columns_column, values=values_column)
    return pivot_table

In [10]:
def add_changes_by_week(weekly_data_frame, outcome_column):

    for column in weekly_data_frame.columns[1:]:
        # Calculate the difference between each row and the previous row
        if outcome_column not in column.lower(): #want to leave out the outcome column
            diff = weekly_data_frame[column].diff()
            
            # Create a new column with the original column name and "delta"
            new_column_name = column + "_delta"
            
            column_index = weekly_data_frame.columns.get_loc(column)
            
            # Insert the new column just after the original column
            weekly_data_frame.insert(column_index + 1, new_column_name, diff)
            weekly_data_frame[new_column_name] = diff
    return weekly_data_frame

# Naive classifier bootstrapping 
- Cannot bootstrap - single, binary predictor 

# Needed data

In [17]:
param_grid = {
    'criterion': ['gini'],#,  'entropy'],
    'max_depth': np.arange(1, 10),
    'min_samples_split':  np.arange(2, 300), #[100, 200, 300, 400, 500], #np.arange(50, 200),
    'min_samples_leaf':  np.arange(2, 400)}#, #100, 200, 300, 400, 500], #np.arange(500, 200)
    #'ccp_alpha': np.arange(0.0001, 0.0035, 0.0001) }


# Create the Decision Tree classifier
cv = RepeatedStratifiedKFold(n_splits=10,  n_repeats=10,random_state=1) ## 10-fold cross validations


# Try CDC Classifier 

In [56]:
data_by_HSA = pd.read_csv('/Users/rem76/Documents/COVID_projections/hsa_time_data_all_dates.csv')
data_by_HSA['health_service_area_number']
data_by_HSA['health_service_area']
#data_by_HSA['HSA_ID'] = data_by_HSA['health_service_area_number'].astype(str) + '' + data_by_HSA['health_service_area'].apply(lambda x: x.split()[0])
data_by_HSA.rename(columns={'health_service_area_number': 'HSA_ID'}, inplace=True)

data_by_HSA['beds_over_15_100k'] = (data_by_HSA['beds_weekly'] > 15)*1

# remove HSAs that have missing data in specific columns

data_by_HSA = data_by_HSA.dropna(subset=['admits_weekly', 'deaths_weekly', 'cases_weekly', 'icu_weekly', 'beds_weekly', 'perc_covid'])

for i, week in enumerate(data_by_HSA['date'].unique()):
    data_by_HSA.loc[data_by_HSA['date'] == week, 'week'] = i

    ## pivot 
data_by_HSA_cases = pivot_data_by_HSA(data_by_HSA, 'week', 'HSA_ID', 'cases_weekly')
data_by_HSA_admissions = pivot_data_by_HSA(data_by_HSA, 'week', 'HSA_ID', 'admits_weekly')
data_by_HSA_percent_beds = pivot_data_by_HSA(data_by_HSA, 'week', 'HSA_ID', 'perc_covid')
data_by_HSA_over_15_100k = pivot_data_by_HSA(data_by_HSA, 'week', 'HSA_ID', 'beds_over_15_100k')

## merge 
data_by_HSA_cases_admits = merge_and_rename_data(data_by_HSA_cases, data_by_HSA_admissions,'week','cases', 'admits')
data_by_HSA_admits_perc_outcome = merge_and_rename_data(data_by_HSA_percent_beds, data_by_HSA_over_15_100k,'week','perc_covid', 'beds_over_15_100k')
data_by_HSA_cases_admits_perc_outcome= pd.merge(data_by_HSA_cases_admits, data_by_HSA_admits_perc_outcome, on='week')


data_by_HSA_cases_admits_perc_outcome = data_by_HSA_cases_admits_perc_outcome.reset_index()
data_by_HSA_cases_admits_perc_outcome.columns = data_by_HSA_cases_admits_perc_outcome.columns.str.replace(',', '')

categories_for_subsetting = ['cases', 'admits','perc_covid', 'beds_over_15_100k']
num_of_weeks = len(data_by_HSA_cases_admits_perc_outcome)
column_names = create_column_names(categories_for_subsetting, num_of_weeks)

all_HSA_ID_weekly_data = create_collated_weekly_data(data_by_HSA_cases_admits_perc_outcome, data_by_HSA, categories_for_subsetting, 'HSA_ID', column_names)

weights_df = data_by_HSA[data_by_HSA['HSA_ID'].isin(all_HSA_ID_weekly_data['HSA_ID'])][['HSA_ID','weight_alt']]
weights_df = weights_df.rename(columns = {'HSA_ID': 'HSA_ID', 'weight_alt':'weight'})
weights_df = weights_df.drop_duplicates()
weights_df['weight'].unique()
all_HSA_ID_weekly_data = all_HSA_ID_weekly_data.join(weights_df['weight'])

/var/folders/d4/chxwf4hs5kq7ttsp56s64z65mjk3qj/T/ipykernel_1472/1805026620.py:1: DtypeWarning: Columns (41,43,44,45,46,50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  data_by_HSA = pd.read_csv('/Users/rem76/Documents/COVID_projections/hsa_time_data_all_dates.csv')


Optimized classifier

In [59]:
X_train, y_train, weights, missing_data_train_HSA = prep_training_test_data(all_HSA_ID_weekly_data,  no_weeks = range(1, int(123*2/3) + 1), weeks_in_future = 3,   geography = 'HSA_ID', weight_col = 'weight', keep_output = False)

X_test, y_test, weights_test, missing_data_test_HSA = prep_training_test_data(all_HSA_ID_weekly_data,  no_weeks = range(int(123*2/3) + 1, 120), weeks_in_future = 3,   geography = 'HSA_ID',  weight_col = 'weight', keep_output = False)
weights = weights[0].to_numpy()

In [23]:
CDC_exact = pickle.load(open("/Users/rem76/Documents/COVID_projections/COVID_forecasting/CDC_optimized_exact_auroc_0.8269_pruned.sav", 'rb'))
bootstrap_no_dev(iterations =range(0,100), clf = CDC_exact,  param_grid = param_grid,  cv = cv , iterations_param_search = 10,data = all_HSA_ID_weekly_data, model_name = "Optimized_CDC_classifier_exact_boostrap", time_period = 'exact', no_weeks = range(1, int(123*2/3) + 1),keep_output = False, weeks_in_future = 3,   geography = 'HSA_ID', weight_col = 'weight')

In [60]:
calculate_percentiles(iterations = range(0,100), model_name = "Optimized_CDC_classifier_exact_boostrap", ROC_actual = 0.8269, accuracy_actual = 0.758, sensitivity_actual = 0.757, specificity_actual = 0.762, ppv_actual = 0.870, npv_actual = 0.599, X_test = X_test, y_test =y_test)

(0.8237764055669351,
 0.8713994771420005,
 0.7432406617945595,
 0.7911488699418054,
 0.7204712637938783,
 0.8187186298497029,
 0.72609098541295,
 0.8105410851086158,
 0.8610524852530571,
 0.8916781952553877,
 0.5711349068555243,
 0.6465129179251549)

# Enhanced CDC optimizer

In [62]:
CDC_exact_enhanced = pickle.load(open("/Users/rem76/Documents/COVID_projections/COVID_forecasting/CDC_optimized_exact_enhanced_auroc_0.8280_pruned.sav", 'rb'))
X_train, y_train, weights, missing_data_train_HSA = prep_training_test_data(all_HSA_ID_weekly_data,  no_weeks = range(1, int(123*2/3) + 1), weeks_in_future = 3,   geography = 'HSA_ID', weight_col = 'weight', keep_output = True)

X_test, y_test, weights_test, missing_data_test_HSA = prep_training_test_data(all_HSA_ID_weekly_data,  no_weeks = range(int(123*2/3) + 1, 120), weeks_in_future = 3,   geography = 'HSA_ID',  weight_col = 'weight', keep_output = True)
weights = weights[0].to_numpy()

In [20]:
bootstrap_no_dev(iterations =range(0,100), clf = CDC_exact_enhanced,  param_grid = param_grid,  cv = cv , iterations_param_search = 10,data = all_HSA_ID_weekly_data, model_name = "Optimized_CDC_classifier_enhanced_exact_boostrap", time_period = 'exact', no_weeks = range(1, int(123*2/3) + 1),keep_output = True, weeks_in_future = 3,   geography = 'HSA_ID', weight_col = 'weight')

In [64]:
calculate_percentiles(iterations = range(0,100), model_name = "Optimized_CDC_classifier_enhanced_exact_boostrap", ROC_actual = 0.8280, accuracy_actual = 0.7899, sensitivity_actual =  0.873, specificity_actual = 0.631, ppv_actual = 0.833, npv_actual = 0.702, X_test = X_test, y_test =y_test)

(0.8227005411620747,
 0.8733634891878018,
 0.7850713493030181,
 0.8310738530247667,
 0.8733780396464772,
 0.9924061017626205,
 0.5072775737223214,
 0.6309789064959597,
 0.7974115047131706,
 0.8334923780487804,
 0.7018748248482017,
 0.8160039231066827)

# Full Classifier 

In [12]:
all_HSA_ID_weekly_data = pd.read_csv('/Users/rem76/Documents/COVID_projections/hsa_time_data_all_dates_weekly.csv')
all_HSA_ID_weekly_data.drop('Unnamed: 0', axis =1, inplace = True)

Period 

In [32]:
Full_period = pickle.load(open("/Users/rem76/Documents/COVID_projections/COVID_forecasting/Full_auroc_0.9092_period_pruned.sav", 'rb'))

X_train, y_train, weights, missing_data_train_HSA, HSA_ID_train = prep_training_test_data_period(all_HSA_ID_weekly_data,   no_weeks = range(1, int(123*2/3) + 1), weeks_in_future = 3,   geography = 'HSA_ID', weight_col = 'weight', keep_output = True)

X_test, y_test, weights_test, missing_data_test_HSA, HSA_ID = prep_training_test_data_period(all_HSA_ID_weekly_data,   no_weeks = range(int(123*2/3) + 1, 120), weeks_in_future = 3,   geography = 'HSA_ID',  weight_col = 'weight', keep_output = True)
#weights = weights[0].to_numpy()

In [610]:
training_data = pd.merge(X_train, y_train, left_index=True, right_index=True)
training_data.rename(columns={0: 'y'}, inplace=True)
training_data = pd.merge(training_data, weights, left_index=True, right_index=True)
training_data.rename(columns={0: 'weight'}, inplace=True)
training_data = pd.merge(training_data, HSA_ID_train, left_index=True, right_index=True)
training_data.rename(columns={0: 'HSA_ID'}, inplace=True)
weights = weights[0].to_numpy()

In [25]:
no_iterations_CV = 10
geography_column = 'HSA_ID'  
geo_split = 0.9  
time_period = 'period'  # Choose 'period', 'exact', or 'shifted'
no_weeks_train =  range(1, int(123*2/3) + 1)  # across entire training period
no_weeks_test = range(int(123*2/3) + 1, 120) # across entire training period
weeks_in_future = 3 
weight_col = 'weight'  
keep_output = True  
no_iterations_param = 20  # Replace with the number of iterations for RandomizedSearchCV
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': np.arange(2, 5, 1),
    'min_samples_split': np.arange(200, 2000, 50), #[100, 200, 300, 400, 500], #np.arange(50, 200),
    'min_samples_leaf':  np.arange(200, 2000, 50)} #100, 200, 300, 400, 500], #np.arange(500, 200)
    #'ccp_alpha': np.arange(0.0001, 0.0035, 0.0001) }
clf = DecisionTreeClassifier(random_state=10, class_weight='balanced')
cv = RepeatedStratifiedKFold(n_splits=10,  n_repeats=10,random_state=1)  ## 10-fold cross validations


In [454]:
bootstrap_no_dev(training_data = training_data,iterations =range(0,10), clf = clf,  geography = geography_column, no_iterations_CV = no_iterations_CV, param_grid = param_grid,  cv = cv , iterations_param_search = no_iterations_param, model_name = "Full_classifier_period_boostrap", geo_split=geo_split,  no_weeks_train=no_weeks_train, no_weeks_test=no_weeks_test, weeks_in_future=weeks_in_future, weight_col=weight_col, keep_output=keep_output, time_period=time_period )

0.9313755018441309
0.941065952029627
0.936976241475838
0.9498619741142746
0.9088579950226103
0.9271055625411653
0.9289600647885079
0.9408022402595915
0.9370066506347459
0.9207916839179039
0.9170433404351779
0.9117436022607238
0.9243117393929001
0.908844890320583
0.9165535237291362
0.8888914158807721
0.9245235123959383
0.9298920266038022
0.9156231493528652
0.9205073915962139
0.9079066387887955
0.9091958740336026
0.9292931685994859
0.9284183172466649
0.9287799352006155
0.9090930769896728
0.9253275845812517
0.8969081786087866
0.922526275911093
0.9149927495851237
0.9257409556439729
0.9227604210816309
0.9249236491918119
0.9368443887726468
0.9037813767539247
0.9354429083461342
0.9245557568922623
0.900252373512799
0.9343824643552064
0.9239232568692496
0.9028639968271601
0.9482822459692538
0.9337387844295887
0.9151440288944832
0.9094472151578206
0.8899590295991504
0.9115178795633645
0.924937577746804
0.9272959582208137
0.9160110874615696
0.902638424794241
0.908262754524175
0.9297315150524348
0

In [18]:
calculate_percentiles(iterations = range(1,10), model_name = "Full_classifier_period_boostrap", ROC_actual = 0.909, accuracy_actual = 0.843, sensitivity_actual = 0.847, specificity_actual = 0.828, ppv_actual = 0.952, npv_actual = 0.572, X_test = X_test, y_test =y_test)

(0.931621636931448,
 0.9386890847618555,
 0.8870082555149547,
 0.9619372039518203,
 0.9176128106671168,
 1.0244274441959575,
 0.7114550110978316,
 0.7655441352228103,
 0.923435888481974,
 0.9376971124024269,
 0.6479941659273787,
 0.7259253046584826)

# Troubleshoot

In [19]:
iterations = [1]
geography = 'HSA_ID'
iterations_param_search = 10

In [22]:
    best_params = cross_validation_leave_geo_out_bootstrap(
        training_data_resampled,
        geography_column=geography,
        geo_split=geo_split,
        no_iterations=no_iterations_CV,
        cv=cv,
        classifier=clf,
        param_grid=param_grid,
        no_iterations_param=iterations_param_search,
        no_weeks_train=no_weeks_train,
        no_weeks_test=no_weeks_test,
        weeks_in_future=weeks_in_future,
        weight_col=weight_col,
        keep_output=keep_output,
        time_period=time_period
    )

NameError: name 'training_data_resampled' is not defined

In [602]:
    best_params = {'min_samples_split': 1250,
 'min_samples_leaf': 1300,
 'max_depth': 4,
 'criterion': 'entropy'}

    model = DecisionTreeClassifier(**best_params, random_state=10, class_weight='balanced')
    model_fit = model.fit(X_sample_train, y_sample_train, sample_weight=weights_train)
    
    # Perform cost complexity pruning
    path = model_fit.cost_complexity_pruning_path(X_sample_train, y_sample_train)
    ccp_alphas, impurities = path.ccp_alphas, path.impurities
    clfs = []
    
    for ccp_alpha in ccp_alphas:
        clf = DecisionTreeClassifier(**best_params, random_state=10, ccp_alpha=ccp_alpha, class_weight='balanced')
        clf.fit(X_sample_train, y_sample_train, sample_weight=weights_train)
        clfs.append(clf)
    
    clfs = clfs[:-1]
    ccp_alphas = ccp_alphas[:-1]
    train_scores = [roc_auc_score(y_sample_train, clf.predict_proba(X_sample_train)[:, 1]) for clf in clfs]
    model_pruned = clfs[train_scores.index(max(train_scores))]

    model_fit = model_pruned.fit(X_sample_train, y_sample_train, sample_weight=weights_train)
    
    

In [28]:
model_name = "Full_classifier_period_boostrap"

In [29]:
        bootstrapped_stats_ROC = []
        bootstrapped_stats_accuracy = []
        bootstrapped_stats_sesitivity = []
        bootstrapped_stats_specificity = []
        bootstrapped_stats_ppv = []
        bootstrapped_stats_npv = []


In [34]:
ROC_actual = 0.909
accuracy_actual = 0.843
sensitivity_actual = 0.847
specificity_actual = 0.828
ppv_actual = 0.952
npv_actual = 0.572

In [35]:
for j in iterations:
    
    X_train, y_train, weights, missing_data_train_HSA, HSA_ID_train = prep_training_test_data_period(all_HSA_ID_weekly_data,   no_weeks = range(1, int(123*2/3) + 1), weeks_in_future = 3,   geography = 'HSA_ID', weight_col = 'weight', keep_output = True)

    X_test, y_test, weights_test, missing_data_test_HSA, HSA_ID = prep_training_test_data_period(all_HSA_ID_weekly_data,   no_weeks = range(int(123*2/3) + 1, 120), weeks_in_future = 3,   geography = 'HSA_ID',  weight_col = 'weight', keep_output = True)
#weights = weights[0].to_numpy()
    training_data = pd.merge(X_train, y_train, left_index=True, right_index=True)
    training_data.rename(columns={0: 'y'}, inplace=True)
    training_data = pd.merge(training_data, weights, left_index=True, right_index=True)
    training_data.rename(columns={0: 'weight'}, inplace=True)
    training_data = pd.merge(training_data, HSA_ID_train, left_index=True, right_index=True)
    training_data.rename(columns={0: 'HSA_ID'}, inplace=True)
    weights = weights[0].to_numpy()
    # Sample the training data
    training_data_resampled = training_data.sample(frac=1, replace=True)
    training_data_resampled_copy = training_data_resampled.copy()
    weights_train = training_data_resampled_copy.loc[:, 'weight']
    y_sample_train = training_data_resampled_copy.loc[:, 'y']
    X_sample_train = training_data_resampled_copy.drop(['HSA_ID', 'weight', 'y'], axis=1)

    # Find the best hyperparameters using cross-validation
    best_params = cross_validation_leave_geo_out_bootstrap(
        training_data_resampled,
        geography_column=geography,
        geo_split=geo_split,
        no_iterations=no_iterations_CV,
        cv=cv,
        classifier=clf,
        param_grid=param_grid,
        no_iterations_param=iterations_param_search,
        no_weeks_train=no_weeks_train,
        no_weeks_test=no_weeks_test,
        weeks_in_future=weeks_in_future,
        weight_col=weight_col,
        keep_output=keep_output,
        time_period=time_period
    )

    # Create the Decision Tree classifier with the best hyperparameters
    model = DecisionTreeClassifier(**best_params, random_state=10, class_weight='balanced')
    model_fit = model.fit(X_sample_train, y_sample_train, sample_weight=weights_train)
    print(best_params)
    # Perform cost complexity pruning
    path = model_fit.cost_complexity_pruning_path(X_sample_train, y_sample_train)
    ccp_alphas, impurities = path.ccp_alphas, path.impurities
    clfs = []
    
    for ccp_alpha in ccp_alphas:
        clf = DecisionTreeClassifier(**best_params, random_state=10, ccp_alpha=ccp_alpha, class_weight='balanced')
        clf.fit(X_sample_train, y_sample_train, sample_weight=weights_train)
        clfs.append(clf)
    
    clfs = clfs[:-1]
    ccp_alphas = ccp_alphas[:-1]
    train_scores = [roc_auc_score(y_sample_train, clf.predict_proba(X_sample_train)[:, 1]) for clf in clfs]
    model_pruned = clfs[train_scores.index(max(train_scores))]

    model_fit = model_pruned.fit(X_sample_train, y_sample_train, sample_weight=weights_train)
    print(model_fit)
    # Save the model and related data
    model_name_to_save = model_name + "_" + str(j) + ".sav"
    X_data_name = model_name + "_X_data_" + str(j) + ".csv"
    y_data_name = model_name + "_y_data_" + str(j) + ".csv"
    weights_data_name = model_name + "_weights_" + str(j) + ".csv"
    weights_train.to_csv(weights_data_name, index=False)
    X_sample_train.to_csv(X_data_name,index=False)
    y_sample_train.to_csv(y_data_name, index=False)
    pickle.dump(model_fit, open(model_name_to_save, 'wb'))
    y_pred = model_pruned.predict(X_test)
    y_pred_proba = model_pruned.predict_proba(X_test)

    # Evaluate the accuracy of the model
    accuracy_bootstrap_test_performance = accuracy_score(y_test, y_pred)
    ROC_AUC_bootstrap_test_performance = roc_auc_score(y_test, y_pred_proba[:,1])


    conf_matrix = confusion_matrix(y_test, y_pred)
    # Calculate sensitivity (True Positive Rate)
    sensitivity_actual = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
    print("sensitivity: ", sensitivity_actual)
    # Calculate specificity (True Negative Rate)
    specificity_actual = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
    print("specificity: ", specificity_actual)

    # Calculate PPV (Positive Predictive Value)
    ppv_actual = conf_matrix[1, 1] / (conf_matrix[0, 1] + conf_matrix[1, 1])
    print("ppv: ", ppv_actual)

    # Calculate NPV (Negative Predictive Value)
    npv_actual = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[1, 0])
    print("npv: ", npv_actual)

    sensitivity_bootstrap_test_performance, specificity_bootstrap_test_performance, ppv_bootstrap_test_performance, npv_bootstrap_test_performance = calculate_metrics(confusion_matrix(y_test, y_pred))
        ### (D) Calculate estimate fo variance  by getting (B) - (D) 

    bootstrapped_stats_ROC.append({'Difference': ROC_AUC_bootstrap_test_performance - ROC_actual}) ## according to https://ocw.mit.edu/courses/18-05-introduction-to-probability-and-statistics-spring-2014/resources/mit18_05s14_reading24/
    bootstrapped_stats_accuracy.append({'Difference': accuracy_bootstrap_test_performance - accuracy_actual}) ## according to https://ocw.mit.edu/courses/18-05-introduction-to-probability-and-statistics-spring-2014/resources/mit18_05s14_reading24/
    bootstrapped_stats_sesitivity.append({'Difference': sensitivity_bootstrap_test_performance - sensitivity_actual}) ## according to https://ocw.mit.edu/courses/18-05-introduction-to-probability-and-statistics-spring-2014/resources/mit18_05s14_reading24/
    bootstrapped_stats_specificity.append({'Difference': specificity_bootstrap_test_performance - specificity_actual}) ## according to https://ocw.mit.edu/courses/18-05-introduction-to-probability-and-statistics-spring-2014/resources/mit18_05s14_reading24/
    bootstrapped_stats_ppv.append({'Difference': ppv_bootstrap_test_performance - ppv_actual}) ## according to https://ocw.mit.edu/courses/18-05-introduction-to-probability-and-statistics-spring-2014/resources/mit18_05s14_reading24/
    bootstrapped_stats_npv.append({'Difference': npv_bootstrap_test_performance - npv_actual}) ## according to https://ocw.mit.edu/courses/18-05-introduction-to-probability-and-statistics-spring-2014/resources/mit18_05s14_reading24/


bootstrapped_stats_ROC = pd.DataFrame(bootstrapped_stats_ROC)
bootstrapped_stats_accuracy = pd.DataFrame(bootstrapped_stats_accuracy)
bootstrapped_stats_sesitivity = pd.DataFrame(bootstrapped_stats_sesitivity)
bootstrapped_stats_specificity = pd.DataFrame(bootstrapped_stats_specificity)
bootstrapped_stats_ppv = pd.DataFrame(bootstrapped_stats_ppv)
bootstrapped_stats_npv = pd.DataFrame(bootstrapped_stats_npv)

    ## Step 3: Get percentile
alpha = 0.05

upper_quartile_ROC, lower_quartile_ROC = ROC_actual - np.percentile(bootstrapped_stats_ROC["Difference"], [100 * (1 - alpha / 2.0), 100 * alpha / 2.0])
upper_quartile_accuracy, lower_quartile_accuracy = accuracy_actual - np.percentile(bootstrapped_stats_accuracy["Difference"], [100 * (1 - alpha / 2.0), 100 * alpha / 2.0])
upper_quartile_sensitivity, lower_quartile_sensitivity = sensitivity_actual - np.percentile(bootstrapped_stats_sesitivity["Difference"], [100 * (1 - alpha / 2.0), 100 * alpha / 2.0])
upper_quartile_specificity, lower_quartile_specificity = specificity_actual - np.percentile(bootstrapped_stats_specificity["Difference"], [100 * (1 - alpha / 2.0), 100 * alpha / 2.0])
upper_quartile_ppv, lower_quartile_ppv = ppv_actual - np.percentile(bootstrapped_stats_ppv["Difference"], [100 * (1 - alpha / 2.0), 100 * alpha / 2.0])
upper_quartile_npv, lower_quartile_npv = npv_actual - np.percentile(bootstrapped_stats_npv["Difference"], [100 * (1 - alpha / 2.0), 100 * alpha / 2.0])
 





[{'min_samples_split': 1650, 'min_samples_leaf': 1950, 'max_depth': 2, 'criterion': 'gini'}]
0.9121996263120155
[{'min_samples_split': 1650, 'min_samples_leaf': 1950, 'max_depth': 2, 'criterion': 'gini'}, {'min_samples_split': 1650, 'min_samples_leaf': 1950, 'max_depth': 2, 'criterion': 'gini'}]
0.90166383517969
[{'min_samples_split': 1650, 'min_samples_leaf': 1950, 'max_depth': 2, 'criterion': 'gini'}, {'min_samples_split': 1650, 'min_samples_leaf': 1950, 'max_depth': 2, 'criterion': 'gini'}, {'min_samples_split': 1650, 'min_samples_leaf': 1950, 'max_depth': 2, 'criterion': 'gini'}]
0.9242068842999263
[{'min_samples_split': 1650, 'min_samples_leaf': 1950, 'max_depth': 2, 'criterion': 'gini'}, {'min_samples_split': 1650, 'min_samples_leaf': 1950, 'max_depth': 2, 'criterion': 'gini'}, {'min_samples_split': 1650, 'min_samples_leaf': 1950, 'max_depth': 2, 'criterion': 'gini'}, {'min_samples_split': 1650, 'min_samples_leaf': 1950, 'max_depth': 2, 'criterion': 'gini'}]
0.9283527582098459
[{

In [542]:
model_name_full_classifier = '/Users/rem76/Documents/COVID_projections/COVID_forecasting/Full_auroc_0.9092_period_pruned.sav'
model_fit_full_classifer = pickle.load(open(model_name_full_classifier, 'rb'))


In [543]:
# Make predictions on the test set

y_pred = model_fit_full_classifer.predict(X_test)
y_pred_proba = model_fit_full_classifer.predict_proba(X_test)

# Evaluate the accuracy of the model
accuracy_actual = accuracy_score(y_test, y_pred)
ROC_actual = roc_auc_score(y_test, y_pred_proba[:,1])
print(accuracy_actual)
print(ROC_actual)

conf_matrix = confusion_matrix(y_test, y_pred)
# Calculate sensitivity (True Positive Rate)
sensitivity_actual = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
print("sensitivity: ", sensitivity_actual)
# Calculate specificity (True Negative Rate)
specificity_actual = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
print("specificity: ", specificity_actual)

# Calculate PPV (Positive Predictive Value)
ppv_actual = conf_matrix[1, 1] / (conf_matrix[0, 1] + conf_matrix[1, 1])
print("ppv: ", ppv_actual)

# Calculate NPV (Negative Predictive Value)
npv_actual = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[1, 0])
print("npv: ", npv_actual)

0.8113073487616728
0.9091874097110433
sensitivity:  0.7948436642896325
specificity:  0.8779238518012634
ppv:  0.9634308510638298
npv:  0.5139944022391043


In [526]:
model_name = 'Full_classifier_period_boostrap'

In [567]:
iterations = range(0,10)
feature_names = ['cases','delta_cases', 'deaths', 'delta_deaths', 'admits', 'delta_admits', 'icu', 'delta_icu',  'beds', 'delta_beds', 'perc_covid', 'delta_perc', 'beds_over_15_100k']

In [573]:
X_train, y_train, weights, missing_data_train_HSA, p = prep_training_test_data_period(all_HSA_ID_weekly_data,   no_weeks = range(1, int(123*2/3) + 1), weeks_in_future = 3,   geography = 'HSA_ID', weight_col = 'weight', keep_output = True)

X_test, y_test, weights_test, missing_data_test_HSA, p = prep_training_test_data_period(all_HSA_ID_weekly_data,   no_weeks = range(int(123*2/3) + 1, 120), weeks_in_future = 3,   geography = 'HSA_ID',  weight_col = 'weight', keep_output = True)
weights = weights[0].to_numpy()

In [582]:
model_fit = DecisionTreeClassifier(class_weight='balanced', max_depth=5,
                       min_samples_leaf=200, min_samples_split=100,
                       random_state=10)
model_name_to_load = model_name + "_" + str(j) + ".sav"
model_fit = pickle.load(open(model_name_to_load, 'rb'))

X_sample_train = pd.read_csv('Full_classifier_period_boostrap_X_data_1.csv')
X_sample_train.columns = feature_names

y_sample_train = pd.read_csv('Full_classifier_period_boostrap_y_data_1.csv')
weights_train =  pd.read_csv('Full_classifier_period_boostrap_weights_1.csv')
weights_train = weights_train['weight']
model_fit.fit(X_train, y_train, sample_weight=weights)
y_pred = model_fit.predict(X_test)
y_pred_proba = model_fit.predict_proba(X_test)

# Evaluate the accuracy of the model
accuracy_actual = accuracy_score(y_test, y_pred)
ROC_actual = roc_auc_score(y_test, y_pred_proba[:,1])
print(accuracy_actual)
print(ROC_actual)



0.7984842333197997
0.886033636400372


In [547]:
bootstrapped_stats_ROC = []
bootstrapped_stats_accuracy = []
bootstrapped_stats_sensitivity = []
bootstrapped_stats_specificity = []
bootstrapped_stats_ppv = []
bootstrapped_stats_npv = []

for j in iterations:
    model_name_to_load = model_name + "_" + str(j) + ".sav"
    model_fit = pickle.load(open(model_name_to_load, 'rb'))
    y_bootstrap_predict = model_fit.predict(X_test)
    y_bootstrap_predict_proba = model_fit.predict_proba(X_test)

    ROC_AUC_bootstrap_test_performance = roc_auc_score(y_test, y_bootstrap_predict_proba[:, 1])
    accuracy_bootstrap_test_performance = accuracy_score(y_test, y_bootstrap_predict)

    sensitivity_bootstrap_test_performance, specificity_bootstrap_test_performance, ppv_bootstrap_test_performance, npv_bootstrap_test_performance = calculate_metrics(confusion_matrix(y_test, y_bootstrap_predict))

    # (D) Calculate estimate of variance by getting (B) - (D)
    bootstrapped_stats_ROC.append({'Difference': ROC_AUC_bootstrap_test_performance - ROC_actual})
    bootstrapped_stats_accuracy.append({'Difference': accuracy_bootstrap_test_performance - accuracy_actual})
    bootstrapped_stats_sensitivity.append({'Difference': sensitivity_bootstrap_test_performance - sensitivity_actual})
    bootstrapped_stats_specificity.append({'Difference': specificity_bootstrap_test_performance - specificity_actual})
    bootstrapped_stats_ppv.append({'Difference': ppv_bootstrap_test_performance - ppv_actual})
    bootstrapped_stats_npv.append({'Difference': npv_bootstrap_test_performance - npv_actual})

bootstrapped_stats_ROC = pd.DataFrame(bootstrapped_stats_ROC)
bootstrapped_stats_accuracy = pd.DataFrame(bootstrapped_stats_accuracy)
bootstrapped_stats_sensitivity = pd.DataFrame(bootstrapped_stats_sensitivity)
bootstrapped_stats_specificity = pd.DataFrame(bootstrapped_stats_specificity)
bootstrapped_stats_ppv = pd.DataFrame(bootstrapped_stats_ppv)
bootstrapped_stats_npv = pd.DataFrame(bootstrapped_stats_npv)

# Step 3: Get percentile
alpha = 0.05

upper_quartile_ROC, lower_quartile_ROC = ROC_actual - np.percentile(bootstrapped_stats_ROC["Difference"], [100 * (1 - alpha / 2.0), 100 * alpha / 2.0])
upper_quartile_accuracy, lower_quartile_accuracy = accuracy_actual - np.percentile(bootstrapped_stats_accuracy["Difference"], [100 * (1 - alpha / 2.0), 100 * alpha / 2.0])
upper_quartile_sensitivity, lower_quartile_sensitivity = sensitivity_actual - np.percentile(bootstrapped_stats_sensitivity["Difference"], [100 * (1 - alpha / 2.0), 100 * alpha / 2.0])
upper_quartile_specificity, lower_quartile_specificity = specificity_actual - np.percentile(bootstrapped_stats_specificity["Difference"], [100 * (1 - alpha / 2.0), 100 * alpha / 2.0])
upper_quartile_ppv, lower_quartile_ppv = ppv_actual - np.percentile(bootstrapped_stats_ppv["Difference"], [100 * (1 - alpha / 2.0), 100 * alpha / 2.0])
upper_quartile_npv, lower_quartile_npv = npv_actual - np.percentile(bootstrapped_stats_npv["Difference"], [100 * (1 - alpha / 2.0), 100 * alpha / 2.0])

# Step 4: Get optimization-corrected performance
